In [1]:
%%capture
!pip install -q langchain==0.3.10
!pip install -q langchain-openai==0.2.12
!pip install -q langchain-community==0.3.11
!pip -q install openai==1.55.3
!pip -q install faiss-cpu
!pip -q install tiktoken

In [2]:
import os
import getpass
from typing import Dict, Any

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
# import os
# import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# 🛠️ **Custom Prompt Templates in LangChain:**

- ✂️ **Tailor-Made**: Craft prompts that fit your model like a glove.

- 🎨**Full Control**: Dictate every detail from instructions to formatting.

- 🔌 **Dynamic Inputs**: Plug in specific data as needed for each task.

Create your own by defining inputs and crafting a `format()` method. 📝

📐 Custom templates are like a bespoke suit—made to fit your model's unique requirements.


#🎨 **Custom vs. Default Prompt Templates:**

- ✏️ **Default Templates**: Like using a template for a letter, where you just fill in the blanks.

- 🎭 **Custom Templates**: Like writing a script for a play, where you have the freedom to craft the entire scene.


In [5]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

template = PromptTemplate.from_template("Hello {name}!")

prompt = template.format(name="Harpreet")

print(prompt)

Hello Harpreet!



**Custom templates let you:**

- 🛠️ **Programmatically Craft Prompts**: Build prompts on-the-fly, tailored to the task at hand.

- **Example - `FunctionExplainerPromptTemplate`**:

  - 👨🏽‍💻 **Function Input**: Takes a function directly.

  - 🔍 **Code Inspection**: Uses `inspect` to get the function's source code.

  - 🗣️ **Prompt Assembly**: Creates a prompt to explain the function in plain language.

🗝️ This approach is like having a Swiss Army knife for prompt creation, giving you the tools to construct exactly what you need.


In [7]:
import inspect
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

def get_source_code(function_name):
    """Return the source code of the provided function."""
    # Using the inspect module to get the source code of the function
    return inspect.getsource(function_name)

# Template string for the prompt that will be sent to the language model
PROMPT = """Given the function name and source code, generate an English language explanation of the function.
Function Name: {function_name}

Source Code:
{source_code}

Explanation:
"""

class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input and formats the prompt template to provide the source code of the function."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        # Ensuring that the only input variable is 'function_name'
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        """Format the prompt using the function's name and source code."""
        # Retrieve the source code of the provided function
        source_code = get_source_code(kwargs["function_name"])

        # Format the PROMPT string using the function name and its source code
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__,
            source_code=source_code
        )
        return prompt

    def _prompt_type(self) -> str:
        """Return the type of prompt."""
        return "function-explainer"

C:\Users\Sourav Banerjee\AppData\Local\Temp\ipykernel_15488\601068411.py:23: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("input_variables")


In [8]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

In [9]:
type(fn_explainer)

__main__.FunctionExplainerPromptTemplate

In [10]:
# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)

print(prompt)

Given the function name and source code, generate an English language explanation of the function.
Function Name: get_source_code

Source Code:
def get_source_code(function_name):
    """Return the source code of the provided function."""
    # Using the inspect module to get the source code of the function
    return inspect.getsource(function_name)


Explanation:



In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

response = llm.invoke(prompt)

print(response.content)

This function is called get_source_code and it takes in a parameter called function_name. It uses the inspect module to retrieve the source code of the provided function and then returns that source code as a string. This can be useful for examining the code of a specific function within a program.


In [14]:
from langchain_core.output_parsers import StrOutputParser

llm_chaim = fn_explainer | llm | StrOutputParser()

for chunk in llm_chaim.stream({"function_name":get_source_code}):
  print(chunk, end="", flush=True)

The function "get_source_code" takes a function name as input and returns the source code of that function. It uses the inspect module to retrieve the source code of the provided function and then returns it as a string. This can be useful for debugging or for dynamically accessing and analyzing the source code of a given function.

# Another example


In [10]:
# Template string for the algorithm optimization prompt
ALGO_PROMPT = """Given the algorithm below, suggest optimizations or potential \
improvements, and return the optimized code.

Algorithm Name: {algorithm_name}

Source Code:
{source_code}

Suggestions:
"""

class AlgorithmOptimizerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes an algorithm as input and formats the prompt template to request optimization suggestions."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "algorithm_function" not in v:
            raise ValueError("algorithm_function must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        """Format the prompt using the algorithm's name and source code."""

        # Retrieve the source code of the provided algorithm
        source_code = get_source_code(kwargs["algorithm_function"])

        # Format the ALGO_PROMPT string using the algorithm name and its source code
        prompt = ALGO_PROMPT.format(
            algorithm_name=kwargs["algorithm_function"].__name__,
            source_code=source_code
        )
        return prompt

    def _prompt_type(self) -> str:
        """Return the type of prompt."""
        return "algorithm-optimizer"


In [11]:
def recursive_factorial(n: int) -> int:
    """Calculate factorial of a number using recursion."""
    if n == 0:
        return 1
    else:
        return n * recursive_factorial(n-1)


In [12]:
# Instantiate the AlgorithmOptimizerPromptTemplate with the appropriate input variable
algo_optimizer = AlgorithmOptimizerPromptTemplate(input_variables=["algorithm_function"])

# Generate a prompt for the function "recursive_factorial"
prompt = algo_optimizer.format(algorithm_function=recursive_factorial)

print(prompt)

Given the algorithm below, suggest optimizations or potential improvements, and return the optimized code.

Algorithm Name: recursive_factorial

Source Code:
def recursive_factorial(n: int) -> int:
    """Calculate factorial of a number using recursion."""
    if n == 0:
        return 1
    else:
        return n * recursive_factorial(n-1)


Suggestions:



In [13]:
result = llm.invoke(prompt)

print(result.content)

1. Use memoization to store the results of previously calculated factorials to avoid redundant calculations.
2. Add input validation to handle negative numbers or non-integer inputs.

Optimized Code:
from functools import lru_cache

@lru_cache(maxsize=None)
def recursive_factorial(n: int) -> int:
    """Calculate factorial of a number using recursion and memoization."""
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer")
    if n == 0:
        return 1
    else:
        return n * recursive_factorial(n-1)


In [14]:
llm_chain = algo_optimizer | llm | StrOutputParser()

for chunk in llm_chain.stream({"algorithm_function":recursive_factorial}):
  print(chunk, end="", flush=True)

1. Use memoization to store the results of previously calculated factorials to avoid redundant calculations.
2. Implement error handling to handle negative input values.
3. Use a loop-based approach for better performance and reduced stack overhead, especially for large input values.

Optimized Code:

def recursive_factorial(n: int) -> int:
    """Calculate factorial of a number using recursion with memoization."""
    if n < 0:
        raise ValueError("Input value must be non-negative")
    memo = {}
    return factorial_helper(n, memo)

def factorial_helper(n: int, memo: dict) -> int:
    if n in memo:
        return memo[n]
    if n == 0:
        return 1
    else:
        result = n * factorial_helper(n-1, memo)
        memo[n] = result
        return result

This optimized code uses memoization to store previously calculated factorials and also includes error handling for negative input values. This approach improves performance and reduces redundant calculations.